# Aula 04 - Avaliação de Sistemas de Recomendação - Exercícios

In [1]:
import pandas as pd
import numpy as np

### Importar base de dados

In [2]:
import wget
!python3 -m wget https://github.com/mmanzato/MBABigData/raw/master/ml-20m-compact.tar.gz
!tar -xvzf ml-20m-compact.tar.gz

100% [....................................................] 65019041 / 65019041
Saved under ml-20m-compact.tar.gz
dataset/
dataset/tags_sample.csv
dataset/._.DS_Store
dataset/.DS_Store
dataset/movies_sample.csv
dataset/._genome-tags.csv
dataset/genome-tags.csv
dataset/._ml-youtube.csv
dataset/ml-youtube.csv
dataset/._genome-scores.csv
dataset/genome-scores.csv
dataset/ratings_sample.csv


In [3]:
movies = pd.read_csv('./dataset/movies_sample.csv')
ratings = pd.read_csv('./dataset/ratings_sample.csv')
df = ratings[['userId', 'movieId', 'rating']]
df = df.merge(movies[['movieId', 'title']])
df

,userId,movieId,rating,title
0,11,7481,5.0,Enemy Mine (1985)
1,11,1046,4.5,Beautiful Thing (1996)
2,11,616,4.0,"Aristocats, The (1970)"
3,11,3535,2.0,American Psycho (2000)
4,11,5669,5.0,Bowling for Columbine (2002)
...,...,...,...,...
190616,138493,288,5.0,Natural Born Killers (1994)
190617,138493,1748,5.0,Dark City (1998)
190618,138493,616,4.0,"Aristocats, The (1970)"
190619,138493,1597,4.5,Conspiracy Theory (1997)


In [4]:
movies_genres = movies.drop('genres', axis=1).join(movies.genres.str.split('|', expand=True)
             .stack().reset_index(drop=True, level=1).rename('genre'))
movies_genres.head()

,movieId,title,genre
0,30,Shanghai Triad (Yao a yao yao dao waipo qiao) ...,Crime
0,30,Shanghai Triad (Yao a yao yao dao waipo qiao) ...,Drama
1,31,Dangerous Minds (1995),Drama
2,37,Across the Sea of Time (1995),Documentary
2,37,Across the Sea of Time (1995),IMAX


In [5]:
movies_tags = pd.read_csv('./dataset/tags_sample.csv')
movies_tags.head()

,userId,movieId,tag,timestamp_y
0,279,916,Gregory Peck,1329962459
1,279,916,need to own,1329962471
2,279,916,romantic comedy,1329962476
3,279,916,Rome,1329962490
4,279,916,royalty,1329962474


In [6]:
map_users = {user: idx for idx, user in enumerate(df.userId.unique())}
map_items = {item: idx for idx, item in enumerate(df.movieId.unique())}

df['userId'] = df['userId'].map(map_users)
df['movieId'] = df['movieId'].map(map_items)

movies_tags['userId'] = movies_tags['userId'].map(map_users)
movies_tags['movieId'] = movies_tags['movieId'].map(map_items)
movies_tags.dropna(inplace=True)
movies_tags['movieId'] = movies_tags.movieId.astype(int)
movies_tags['userId'] = movies_tags.userId.astype(int)

movies_genres['movieId'] = movies_genres['movieId'].map(map_items)
movies_genres.dropna(inplace=True)
movies_genres['movieId'] = movies_genres.movieId.astype(int)

map_title = {}
for _, row in df.iterrows():
    map_title[row.movieId] = row.title


In [7]:
movies_genres[['movieId', 'genre']].to_csv('items_genres.dat', index=False, sep='\t', header=False)
movies_tags[['movieId', 'tag']].to_csv('items_tags.dat', index=False, sep='\t', header=False)

### Divisão da base em treino e teste

In [8]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=.2, random_state=2)
train.to_csv('train.dat', index=False, header=False, sep='\t')
test.to_csv('test.dat', index=False, header=False, sep='\t')

### Exemplos de utilização do CaseRecommender

In [9]:
from caserec.recommenders.rating_prediction.item_attribute_knn import ItemAttributeKNN

ItemAttributeKNN('train.dat', 'test.dat', metadata_file='items_genres.dat', k_neighbors=10, as_similar_first=True).compute()
ItemAttributeKNN('train.dat', 'test.dat', metadata_file='items_tags.dat', k_neighbors=10, as_similar_first=True).compute()

[Case Recommender: Rating Prediction > Item Attribute KNN Algorithm]

train data:: 11090 users and 405 items (152496 interactions) | sparsity:: 96.60%
test data:: 10571 users and 331 items (38125 interactions) | sparsity:: 98.91%

training_time:: 3.867399 sec
>> metadata:: 417 items and 20 metadata (890 interactions) | sparsity:: 89.33%
prediction_time:: 0.412804 sec
Eval:: MAE: 0.73347 RMSE: 0.964352 
[Case Recommender: Rating Prediction > Item Attribute KNN Algorithm]

train data:: 11090 users and 405 items (152496 interactions) | sparsity:: 96.60%
test data:: 10571 users and 331 items (38125 interactions) | sparsity:: 98.91%

training_time:: 4.106449 sec
>> metadata:: 231 items and 1979 metadata (6274 interactions) | sparsity:: 98.63%
prediction_time:: 0.502195 sec
Eval:: MAE: 0.765116 RMSE: 1.006911 


***Exercício 01:*** Verifique o efeito no RSME ao aumentar o número de vizinhos do algoritmo ItemAttributeKNN com usando gêneros. Explique.

In [24]:
import numpy as np
from collections import defaultdict

# 1) Mapa de gêneros por item (IDs já mapeados)
genres_by_item = defaultdict(list)
for _, row in movies_genres[['movieId','genre']].iterrows():
    genres_by_item[int(row.movieId)].append(str(row.genre))

# 2) Similaridade Jaccard entre itens
def jaccard(a, b):
    A, B = set(a), set(b)
    return len(A & B) / len(A | B) if (A or B) else 0.0

n_items = int(df.movieId.max()) + 1
S_gen = np.zeros((n_items, n_items), dtype=np.float32)

for i in range(n_items):
    Gi = genres_by_item.get(i, [])
    for j in range(i+1, n_items):
        Gj = genres_by_item.get(j, [])
        s = jaccard(Gi, Gj)
        S_gen[i, j] = S_gen[j, i] = s
np.fill_diagonal(S_gen, 1.0)

# 3) Predição KNN (média ponderada)
def predict_knn_gen(uid, iid, k=10):
    user_hist = train[train.userId == uid][['movieId','rating']]
    if user_hist.empty:
        return 3.0  # fallback neutro

    sims = [(S_gen[iid, int(j)], float(r)) for j, r in user_hist.to_numpy()]
    # considere apenas vizinhos com sim > 0
    sims = [(s, r) for s, r in sims if s > 0]
    sims.sort(key=lambda x: x[0], reverse=True)
    sims = sims[:k]

    if not sims:
        return float(user_hist.rating.mean())

    num = sum(s * r for s, r in sims)
    den = sum(s for s, _ in sims)
    return (num / den) if den != 0 else float(user_hist.rating.mean())

# 4) Avaliar para vários valores de k
def mae_rmse(y_true, y_pred):
    y_true = np.asarray(y_true, float); y_pred = np.asarray(y_pred, float)
    mae = np.mean(np.abs(y_true - y_pred))
    rmse = float(np.sqrt(np.mean((y_true - y_pred)**2)))
    return mae, rmse

k_list = [1, 3, 5, 10, 20, 30, 50]
results = []

for k in k_list:
    preds = [predict_knn_gen(u, i, k=k) for u, i in zip(test.userId, test.movieId)]
    mae, rmse = mae_rmse(test.rating, preds)
    results.append((k, mae, rmse))
    print(f"k={k:>2}  |  MAE={mae:.4f}  RMSE={rmse:.4f}")

# Observação/explicação (curta):
print("\nObservação:")
print("- Em geral, aumentar k reduz a variância (mais vizinhos) mas pode aumentar o viés (inclui vizinhos pouco semelhantes).")
print("- Tipicamente o RMSE melhora até um ponto ótimo e depois piora quando k fica grande demais (efeito de diluição).")


k= 1  |  MAE=0.9924  RMSE=1.3255
k= 3  |  MAE=0.8541  RMSE=1.1083
k= 5  |  MAE=0.8236  RMSE=1.0676
k=10  |  MAE=0.8089  RMSE=1.0484
k=20  |  MAE=0.8064  RMSE=1.0453
k=30  |  MAE=0.8062  RMSE=1.0451
k=50  |  MAE=0.8062  RMSE=1.0451

Observação:
- Em geral, aumentar k reduz a variância (mais vizinhos) mas pode aumentar o viés (inclui vizinhos pouco semelhantes).
- Tipicamente o RMSE melhora até um ponto ótimo e depois piora quando k fica grande demais (efeito de diluição).


***Exercício 02:*** Um importante aspecto que pode ser avaliado em Sistemas de Recomendação é a diversidade da lista de recomendações. A métrica Intra-List Similarity (ILS) aplica uma função de similaridade (e.g. Cosseno, Jaccard, Pearson, etc.) entre todos os pares de itens da lista de recomendação, usando seus metadados como gêneros, tags, etc. Mais detalhes sobre essa métrica podem ser encontrados em: https://grouplens.org/site-content/uploads/Improving-WWW-20051.pdf

***a)*** Implemente uma função que calcula a ILS de uma lista de recomendação para um único usuário. Utilize os gêneros de filmes.

In [25]:
from itertools import combinations
import numpy as np

# Reaproveita 'genres_by_item' e 'jaccard' declarados acima

def ils_from_reclist(item_list):
    """
    Calcula ILS (Intra-List Similarity) de uma lista de itens:
    média do Jaccard de gêneros entre todos os pares da lista.
    """
    items = [int(i) for i in item_list if i in genres_by_item]
    if len(items) < 2:
        return 0.0
    sims = []
    for a, b in combinations(items, 2):
        sims.append(jaccard(genres_by_item.get(a, []), genres_by_item.get(b, [])))
    return float(np.mean(sims)) if sims else 0.0

# Exemplo rápido (Top-10 mais populares no treino)
popular = (
    train.groupby('movieId').size().sort_values(ascending=False).head(10).index.tolist()
)
print("ILS (exemplo - 10 populares):", round(ils_from_reclist(popular), 4))


ILS (exemplo - 10 populares): 0.1311


***b)*** Utilize a função que implementou no item (a) para calcular a ILS de todos os usuários da base de dados. Utilize as recomendações geradas pelo algoritmo BPR MF do CaseRecommender.

In [26]:
# ==== BPR-MF + ILS (robusto a variações do construtor) ====
from caserec.recommenders.item_recommendation.bprmf import BprMF
from collections import defaultdict
from itertools import combinations
import numpy as np
import inspect

# --- se ainda não tiver as funções/estruturas para ILS, crie-as:
# genres_by_item: {movieId_interno: [generos]}, jaccard(), ils_from_reclist()
try:
    genres_by_item  # já definido antes?
except NameError:
    from collections import defaultdict as _dd
    genres_by_item = _dd(list)
    for _, row in movies_genres[['movieId','genre']].iterrows():
        genres_by_item[int(row.movieId)].append(str(row.genre))

def jaccard(a, b):
    A, B = set(a), set(b)
    return len(A & B) / len(A | B) if (A or B) else 0.0

def ils_from_reclist(item_list):
    items = [int(i) for i in item_list if i in genres_by_item]
    if len(items) < 2:
        return 0.0
    sims = [jaccard(genres_by_item[a], genres_by_item[b]) for a, b in combinations(items, 2)]
    return float(np.mean(sims)) if sims else 0.0

# --- 1) Rodar BPR-MF para gerar recomendações (detectando kwargs suportados)
rank_output = "ranking_bprmf.dat"

# kwargs básicos
kwargs = dict(
    train_file='train.dat',
    test_file='test.dat',     # opcional, mas útil para métricas internas do lib
    output_file=rank_output,
    factors=50,
    learn_rate=0.05,
    max_iter=50
)

# detecta se há algum nome de regularização aceito; se houver, define
sig = inspect.signature(BprMF.__init__)
possible_reg_names = ['lmbda', 'lambda_reg', 'reg', 'regularization']
for name in possible_reg_names:
    if name in sig.parameters:
        kwargs[name] = 0.01
        break  # usa o primeiro que encontrar

# instancia e treina
bpr = BprMF(**{k: v for k, v in kwargs.items() if k in sig.parameters})
bpr.compute()
print(f"Arquivo de ranking gerado: {rank_output}")

# --- 2) Ler o ranking gerado (formato: user \\t item \\t score)
rank = []
with open(rank_output, 'r') as f:
    for line in f:
        parts = line.strip().split('\t')
        if len(parts) < 2:
            continue
        try:
            u = int(parts[0]); i = int(parts[1])
            s = float(parts[2]) if len(parts) > 2 else 0.0
            rank.append((u, i, s))
        except:
            continue

if not rank:
    raise RuntimeError("Ranking vazio. Verifique se train.dat/test.dat existem e possuem interações suficientes.")

# --- 3) Montar top-N por usuário e calcular ILS
topN = 10
rec_by_user = defaultdict(list)
for u, i, s in rank:
    rec_by_user[u].append((i, s))

ils_by_user = {}
for u, lst in rec_by_user.items():
    lst.sort(key=lambda x: x[1], reverse=True)
    top_items = [i for i, _ in lst[:topN]]
    ils_by_user[u] = ils_from_reclist(top_items)

# --- 4) Estatísticas gerais de diversidade (ILS)
ils_values = list(ils_by_user.values())
print(f"Usuários com ILS calculada: {len(ils_values)}" if ils_values else "Nenhum ILS calculado.")
if ils_values:
    print(f"ILS média (Top-{topN}): {np.mean(ils_values):.4f}")
    print(f"ILS mediana (Top-{topN}): {np.median(ils_values):.4f}")
    print(f"ILS min/max (Top-{topN}): {np.min(ils_values):.4f} / {np.max(ils_values):.4f}")


[Case Recommender: Item Recommendation > BPRMF]

train data:: 11090 users and 405 items (152496 interactions) | sparsity:: 96.60%
test data:: 10571 users and 331 items (38125 interactions) | sparsity:: 98.91%

training_time:: 87.748596 sec
prediction_time:: 1.655691 sec


Eval:: PREC@1: 0.418598 PREC@3: 0.30694 PREC@5: 0.253524 PREC@10: 0.18458 RECALL@1: 0.135705 RECALL@3: 0.284621 RECALL@5: 0.383243 RECALL@10: 0.543586 MAP@1: 0.418598 MAP@3: 0.512936 MAP@5: 0.515685 MAP@10: 0.487484 NDCG@1: 0.418598 NDCG@3: 0.602217 NDCG@5: 0.619377 NDCG@10: 0.612106 
Arquivo de ranking gerado: ranking_bprmf.dat
Usuários com ILS calculada: 11090
ILS média (Top-10): 0.1755
ILS mediana (Top-10): 0.1728
ILS min/max (Top-10): 0.0626 / 0.4459
